<a href="https://colab.research.google.com/github/Kathy42xu/Blockhouse_quant_assessment/blob/main/Auto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.makedirs('movielens', exist_ok=True)


!wget -O movielens/ml-32m.zip https://files.grouplens.org/datasets/movielens/ml-32m.zip

!unzip movielens/ml-32m.zip -d movielens/

import pandas as pd

# Load the ratings data
ratings = pd.read_csv('movielens/ml-32m/ratings.csv')
print("Ratings Data:")
print(ratings.head())

# Load the movies data
movies = pd.read_csv('movielens/ml-32m/movies.csv')
print("\nMovies Data:")
print(movies.head())

# Load the tags data
tags = pd.read_csv('movielens/ml-32m/tags.csv')
print("\nTags Data:")
print(tags.head())

# Load the links data
links = pd.read_csv('movielens/ml-32m/links.csv')
print("\nLinks Data:")
print(links.head())


--2025-01-17 15:20:06--  https://files.grouplens.org/datasets/movielens/ml-32m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238950008 (228M) [application/zip]
Saving to: ‘movielens/ml-32m.zip’

movielens/ml-32m.zi 100%[===================>] 227.88M  16.6MB/s    in 15s     

2025-01-17 15:20:22 (15.2 MB/s) - ‘movielens/ml-32m.zip’ saved [238950008/238950008]

Archive:  movielens/ml-32m.zip
   creating: movielens/ml-32m/
  inflating: movielens/ml-32m/tags.csv  
  inflating: movielens/ml-32m/links.csv  
  inflating: movielens/ml-32m/README.txt  
  inflating: movielens/ml-32m/checksums.txt  
  inflating: movielens/ml-32m/ratings.csv  
  inflating: movielens/ml-32m/movies.csv  
Ratings Data:
   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load datasets
ratings = pd.read_csv('movielens/ml-32m/ratings.csv')
movies = pd.read_csv('movielens/ml-32m/movies.csv')

# Merge ratings with movies to filter based on titles
merged_data = ratings.merge(movies, on="movieId")
all_ratings = merged_data[
    merged_data['title'].str.endswith(tuple(f"({year})" for year in range(2020, 2025)))
]

# Display filtered data
print(f"Amount of data after filtering: {len(all_ratings)}")
print(all_ratings.head())

# Create user-item interaction matrix
user_item_matrix = all_ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Split into train and test sets
train_data, test_data = train_test_split(user_item_matrix, test_size=0.2, random_state=42)

Amount of data after filtering: 273932
      userId  movieId  rating   timestamp  \
2237      22   235509     2.5  1611070726   
2238      22   247150     3.0  1622483323   
2239      22   278420     4.0  1668895485   
2240      22   279412     3.5  1668895505   
2241      22   280236     4.0  1674334098   

                                    title                   genres  
2237              Outside the Wire (2021)   Action|Sci-Fi|Thriller  
2238                      Stowaway (2021)             Drama|Sci-Fi  
2239  Weird: The Al Yankovic Story (2022)             Comedy|Drama  
2240                Enola Holmes 2 (2022)  Adventure|Crime|Mystery  
2241                      Spirited (2022)                   Comedy  


In [17]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

class AutoRec:
    def __init__(self, num_items, hidden_neurons, learning_rate, lambda_value):
        self.num_items = num_items
        self.hidden_neurons = hidden_neurons
        self.learning_rate = learning_rate
        self.lambda_value = lambda_value

        self.input_R = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.num_items])
        self.input_mask_R = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.num_items])

        self.build_model()

    def build_model(self):
      with tf.compat.v1.variable_scope("AutoRec", reuse=tf.compat.v1.AUTO_REUSE):
        V = tf.compat.v1.get_variable("V", shape=[self.num_items, self.hidden_neurons],
                                      initializer=tf.compat.v1.truncated_normal_initializer(mean=0, stddev=0.03))
        W = tf.compat.v1.get_variable("W", shape=[self.hidden_neurons, self.num_items],
                                      initializer=tf.compat.v1.truncated_normal_initializer(mean=0, stddev=0.03))
        mu = tf.compat.v1.get_variable("mu", initializer=tf.zeros(self.hidden_neurons))
        b = tf.compat.v1.get_variable("b", initializer=tf.zeros(self.num_items))

        # Encoder
        self.encoder = tf.nn.sigmoid(tf.matmul(self.input_R, V) + mu)

        # Decoder
        self.decoder = tf.matmul(self.encoder, W) + b

        # Loss
        pre_rec_cost = tf.multiply(self.input_R - self.decoder, self.input_mask_R)
        rec_cost = tf.reduce_sum(tf.square(pre_rec_cost))
        reg_cost = self.lambda_value * 0.5 * (tf.nn.l2_loss(W) + tf.nn.l2_loss(V))

        self.loss = rec_cost + reg_cost
        self.optimizer = tf.compat.v1.train.AdamOptimizer(self.learning_rate).minimize(self.loss)


    def train(self, train_data, mask, epochs, batch_size):
        session = tf.compat.v1.Session()
        session.run(tf.compat.v1.global_variables_initializer())

        for epoch in range(epochs):
            total_loss = 0
            for i in range(0, len(train_data), batch_size):
                batch_data = train_data[i:i + batch_size]
                batch_mask = mask[i:i + batch_size]
                feed_dict = {self.input_R: batch_data, self.input_mask_R: batch_mask}
                _, batch_loss = session.run([self.optimizer, self.loss], feed_dict=feed_dict)
                total_loss += batch_loss
            print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_data)}")

        self.session = session

    def predict(self, test_data):
        feed_dict = {self.input_R: test_data}
        return self.session.run(self.decoder, feed_dict=feed_dict)


In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Prepare mask for training
train_mask = (train_data > 0).astype(float)

# Instantiate and train AutoRec
num_items = user_item_matrix.shape[1]
hidden_neurons = 500
learning_rate = 0.001
lambda_value = 0.1
batch_size = 128
epochs = 50

autorec = AutoRec(num_items, hidden_neurons, learning_rate, lambda_value)
autorec.train(train_data.values, train_mask.values, epochs, batch_size)

# Predict on test set
test_predictions = autorec.predict(test_data.values)

# Evaluate
test_mask = (test_data > 0).astype(float)
true_ratings = test_data.values[test_mask.values > 0]
predicted_ratings = test_predictions[test_mask.values > 0]

rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
mae = mean_absolute_error(true_ratings, predicted_ratings)

print(f"RMSE: {rmse}, MAE: {mae}")


Epoch 1, Loss: 26.37155637925129
Epoch 2, Loss: 12.805680086105973
Epoch 3, Loss: 10.999481268365091
Epoch 4, Loss: 9.683601651131806
Epoch 5, Loss: 8.609995806036551
Epoch 6, Loss: 7.521076610986626
Epoch 7, Loss: 6.739737504875856
Epoch 8, Loss: 5.9795762264517025
Epoch 9, Loss: 5.338915321558594
Epoch 10, Loss: 4.831098768226595
Epoch 11, Loss: 4.455601313126209
Epoch 12, Loss: 4.113117302503167
Epoch 13, Loss: 3.8243950294284157
Epoch 14, Loss: 3.5014272708840943
Epoch 15, Loss: 3.2458047420925125
Epoch 16, Loss: 3.048075142000611
Epoch 17, Loss: 2.926522169840674
Epoch 18, Loss: 2.764923915855626
Epoch 19, Loss: 2.6715614092564617
Epoch 20, Loss: 2.563806288508828
Epoch 21, Loss: 2.4992395071704805
Epoch 22, Loss: 2.45005186091061
Epoch 23, Loss: 2.4266133727750665
Epoch 24, Loss: 2.4261550676049137
Epoch 25, Loss: 2.4270615782118297
Epoch 26, Loss: 2.4655262542317495
Epoch 27, Loss: 2.4852877210388065
Epoch 28, Loss: 2.4683436579136724
Epoch 29, Loss: 2.435510638093349
Epoch 30, 

#Try new dataset

In [1]:
import torch
torch.cuda.empty_cache()  # Clear GPU cache

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import imp
import numpy as np

from zipfile import ZipFile
try:
    from urllib.request import urlretrieve
except ImportError:  # Python 2 compat
    from urllib import urlretrieve

# this line need to be changed if not on colab:
data_folder = '/content/'


ML_1M_URL = "https://files.grouplens.org/datasets/movielens/ml-32m.zip"
ML_1M_FILENAME = op.join(data_folder,ML_1M_URL.rsplit('/', 1)[1])
ML_1M_FOLDER = op.join(data_folder,'ml-32m')

In [3]:
if not op.exists(ML_1M_FILENAME):
    print('Downloading %s to %s...' % (ML_1M_URL, ML_1M_FILENAME))
    urlretrieve(ML_1M_URL, ML_1M_FILENAME)

if not op.exists(ML_1M_FOLDER):
    print('Extracting %s to %s...' % (ML_1M_FILENAME, ML_1M_FOLDER))
    ZipFile(ML_1M_FILENAME).extractall(data_folder)

In [4]:
import os

# Path to the extracted dataset folder
dataset_path = '/content/ml-32m'  # Adjust this path if needed

# List all files in the folder
files = os.listdir(dataset_path)
print("Files in the dataset:")
for file in files:
    print(file)

Files in the dataset:
movies.csv
checksums.txt
README.txt
tags.csv
links.csv
ratings.csv


In [5]:
import pandas as pd

ratings = pd.read_csv('/content/ml-32m/ratings.csv')
print(ratings.head())

   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       29     2.0  943230976
3       1       30     5.0  944249077
4       1       32     5.0  943228858


In [6]:
movies = pd.read_csv('/content/ml-32m/movies.csv')
print(movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [7]:
# Ensure `movieId` in both datasets is of the same type
ratings['movieId'] = ratings['movieId'].astype(int)
movies['movieId'] = movies['movieId'].astype(int)

# Merge the ratings and movies datasets on the common column 'movieId'
all_ratings = ratings.merge(movies, on='movieId')

# Preview the merged data
print(all_ratings.head())

   userId  movieId  rating  timestamp  \
0       1       17     4.0  944249077   
1       1       25     1.0  944250228   
2       1       29     2.0  943230976   
3       1       30     5.0  944249077   
4       1       32     5.0  943228858   

                                               title  \
0                       Sense and Sensibility (1995)   
1                           Leaving Las Vegas (1995)   
2  City of Lost Children, The (Cité des enfants p...   
3  Shanghai Triad (Yao a yao yao dao waipo qiao) ...   
4          Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   

                                   genres  
0                           Drama|Romance  
1                           Drama|Romance  
2  Adventure|Drama|Fantasy|Mystery|Sci-Fi  
3                             Crime|Drama  
4                 Mystery|Sci-Fi|Thriller  


In [13]:
# Define the cutoff UNIX timestamp for 2024 June
cutoff_timestamp_2022 = 1692302639

# Filter the dataset for data after 2024
new_data = all_ratings[all_ratings['timestamp'] >= cutoff_timestamp_2022].copy()

# Count the number of records
num_records_after_2021 = len(new_data)

print(f"Number of records after 2021: {num_records_after_2021}")

Number of records after 2021: 168882


In [14]:
# Create User-Item Interaction Matrix
user_item_matrix = new_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# 查看生成的交互矩阵
print("User-Item Interaction Matrix:")
print(user_item_matrix.head())


User-Item Interaction Matrix:
movieId  1       2       3       4       5       6       7       9       \
userId                                                                    
28          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
75          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
79          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
144         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
178         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  10      11      ...  292617  292619  292625  292627  292709  292731  \
userId                   ...                                                   
28          0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
75          0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
79          0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
144         0.0     0.0  ...     0.0     0.0

In [15]:
from sklearn.model_selection import train_test_split

# Split into train and test sets
train_data, test_data = train_test_split(user_item_matrix, test_size=0.2, random_state=42)

# 创建掩码 (Mask) 矩阵，用于标记已评分的数据
train_mask = (train_data > 0).astype(float)
test_mask = (test_data > 0).astype(float)

print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")


Train data shape: (2936, 17985)
Test data shape: (735, 17985)


In [20]:
# Define hyperparameters
num_items = user_item_matrix.shape[1]
hidden_neurons = 500
learning_rate = 0.001
lambda_value = 0.1
batch_size = 128
epochs = 50

# Instantiate and train the model
autorec = AutoRec(num_items, hidden_neurons, learning_rate, lambda_value)
autorec.train(train_data.values, train_mask.values, epochs, batch_size)

# Predict on test set
test_predictions = autorec.predict(test_data.values)

# Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error

true_ratings = test_data.values[test_mask.values > 0]
predicted_ratings = test_predictions[test_mask.values > 0]

rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
mae = mean_absolute_error(true_ratings, predicted_ratings)

print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}")


Epoch 1, Loss: 241.02028362887432
Epoch 2, Loss: 65.52700755771565
Epoch 3, Loss: 45.44964566347709
Epoch 4, Loss: 39.9240370914137
Epoch 5, Loss: 36.68734865942183
Epoch 6, Loss: 34.12954961376554
Epoch 7, Loss: 31.793474743086897
Epoch 8, Loss: 29.76262554615655
Epoch 9, Loss: 27.96130396039999
Epoch 10, Loss: 26.32609300327561
Epoch 11, Loss: 24.841238120596156
Epoch 12, Loss: 23.432187363627172
Epoch 13, Loss: 22.145386448997893
Epoch 14, Loss: 20.91818224831563
Epoch 15, Loss: 19.798847084149354
Epoch 16, Loss: 18.781331366349306
Epoch 17, Loss: 17.85716126985056
Epoch 18, Loss: 16.958420465038
Epoch 19, Loss: 16.100006186669788
Epoch 20, Loss: 15.303344310791681
Epoch 21, Loss: 14.533219028233832
Epoch 22, Loss: 13.706840411194014
Epoch 23, Loss: 13.059992236727265
Epoch 24, Loss: 12.485667257282975
Epoch 25, Loss: 11.860256621233448
Epoch 26, Loss: 11.357776693817055
Epoch 27, Loss: 11.110774962713673
Epoch 28, Loss: 10.581285377939
Epoch 29, Loss: 10.642947558161348
Epoch 30, L